In [54]:
import cv2
import json
import collections
from pathlib import Path

In [55]:
patterns = ['*.json']
input_dir = Path('/home/trucly/Documents/DATASET/PROJECTS/HEALTH_INSURANCE/all')
output_dir = '/home/trucly/Documents/DATASET/PROJECTS/HEALTH_INSURANCE/after_process/'
paths = []
for pattern in patterns:
    paths += list(input_dir.glob(f'**/{pattern}'))      

In [56]:
for path in paths:
    filename = str(path)
    with open(filename, 'r') as f:
        obj = json.load(f)

    image_name = obj['imagePath']
    image = cv2.imread(str(input_dir.joinpath(image_name)))
    shapes = obj['shapes']    
    ID = {}
    index_id = -1
    for i in range(len(shapes)):
        x = shapes[i]
        label = x['label']
        if 'value' in x:
            value = x['value']
        if 'V_ID' in label:
            if label != 'V_ID':
                ID[label] = value 
            else:
                index_id = i

        if 'DATE' in label:
            obj['shapes'][i]['value'] = "".join(x for x in value if x.isdigit())

        if label == 'V_TL' or label == 'V_ISS':
            obj['shapes'][i]['value'] = "".join(x for x in value if x.isdigit() or x == '/')


    ID = collections.OrderedDict(sorted(ID.items()))
    obj['shapes'][index_id]['value'] = " ".join(x for x in ID.values())

    with open(output_dir + image_name.split('.')[0] + '.json', 'w') as f:
        json.dump(obj, f, indent=2)